In [1]:
import json
from tqdm import tqdm
from multiprocessing.pool import ThreadPool
import pandas as pd

from constants import DATA_DIR, URLS_PICKLE, BIAS_PICKLE, TEXTS_DIR, VAMPIRE_DIR
from util import load_text

In [2]:
# Create dataframe
bias = pd.read_pickle(BIAS_PICKLE)
urls = pd.read_pickle(URLS_PICKLE)
df = urls.merge(bias)

df.head()

,url,filename,domain,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes
0,http://www.dailykos.com/storyonly/2010/5/31/87...,0400971-cbeb4dc21dd10f5966ee4314727252f8.txt,dailykos.com,4884,2.64,Strongly Agrees,https://www.allsides.com/news-source/daily-kos,left,1850,Daily Kos,6734
1,http://www.dailykos.com/storyonly/2010/5/31/87...,0812606-37a92c383f190c185c6f3ef2424111c9.txt,dailykos.com,4884,2.64,Strongly Agrees,https://www.allsides.com/news-source/daily-kos,left,1850,Daily Kos,6734
2,http://www.dailykos.com/storyonly/2010/5/31/87...,0581869-89b8a3fd618318da2a53e17f9782f267.txt,dailykos.com,4884,2.64,Strongly Agrees,https://www.allsides.com/news-source/daily-kos,left,1850,Daily Kos,6734
3,http://www.dailykos.com/storyonly/2010/6/1/871...,0191512-60955e3f214f743e80e16942718fd60e.txt,dailykos.com,4884,2.64,Strongly Agrees,https://www.allsides.com/news-source/daily-kos,left,1850,Daily Kos,6734
4,http://www.dailykos.com/storyonly/2010/6/1/871...,0597616-96f64f32180551e8d61b9fa86d86ce88.txt,dailykos.com,4884,2.64,Strongly Agrees,https://www.allsides.com/news-source/daily-kos,left,1850,Daily Kos,6734


In [3]:
# Load text into dataframe
tqdm.pandas()
df['text'] = df['filename'].progress_apply(lambda filename: load_text(filename))

/home/sam/anaconda3/envs/lm-toxicity/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 2018400/2018400 [06:19<00:00, 5316.87it/s] 


In [ ]:
# Save hdf file
df.to_hdf(VAMPIRE_DIR / 'data.h5', key='df', mode='w')

In [11]:
# Split dataframe
df_shuffled = df.sample(frac=1)
split_1 = int(0.8 * len(df_shuffled))
split_2 = int(0.9 * len(df_shuffled))

split_dfs = df_shuffled[:split_1], df_shuffled[split_1:split_2], df_shuffled[split_2:]
split_paths = VAMPIRE_DIR / 'bias-train.jsonl', VAMPIRE_DIR / 'bias-dev.jsonl', VAMPIRE_DIR / 'bias-test.jsonl'
splits = list(zip(split_dfs, split_paths))

In [12]:
train_df = split_dfs[0]
# df.to_json(orient='records', lines=True)

,url,filename,domain,agree,agree_ratio,agreeance_text,allsides_page,bias,disagree,name,total_votes
528686,http://www.huffingtonpost.com/entry/trump-and-...,0888309-9113732bbf8e6a73038c366bb3ce98fd.txt,huffingtonpost.com,17976,0.934789,Somewhat Disagrees,https://www.allsides.com/news-source/huffpost-...,left,19230,HuffPost,37206
1898068,http://www.theguardian.com/business/2015/dec/3...,0589102-ecdf0d3ee597c53f47ab3d96e14c95bb.txt,theguardian.com,5455,1.603469,Agrees,https://www.allsides.com/news-source/guardian,left-center,3402,The Guardian,8857
127807,http://www.cnn.com/2017/05/20/middleeast/iran-...,0040265-b7499c4918480f2825dbce3c339cd62b.txt,cnn.com,6012,3.192777,Absolutely Agrees,https://www.allsides.com/news-source/cnn-edito...,left,1883,CNN - Editorial,7895
1319572,http://thehill.com/blogs/pundits-blog/energy-e...,0256659-1c77659bf4e84f63a7823a520ed4c036.txt,thehill.com,6506,0.807697,Somewhat Disagrees,https://www.allsides.com/news-source/hill-medi...,center,8055,The Hill,14561
805350,http://www.washingtonpost.com/politics/mitt-ro...,0928833-505dd739e17329e7c48729092f1f4765.txt,washingtonpost.com,23100,1.624358,Agrees,https://www.allsides.com/news-source/washingto...,left-center,14221,Washington Post,37321
...,...,...,...,...,...,...,...,...,...,...,...
608580,http://www.telegraph.co.uk/culture/art/art-new...,0763931-7a4795a6448df038ea4b44c78064499c.txt,telegraph.co.uk,500,1.700680,Agrees,https://www.allsides.com/news-source/telegraph-uk,right-center,294,The Telegraph - UK,794
1080403,http://www.npr.org/2014/03/20/291896473/soarin...,0579881-8932b3f2f59843f60f9a7a7d136ea84c.txt,npr.org,18286,1.380180,Somewhat Agrees,https://www.allsides.com/news-source/npr-media...,center,13249,NPR News,31535
1643318,http://www.theverge.com/2014/8/28/6080779/amaz...,0146549-3971c758d473d0925e6baee9c7494423.txt,theverge.com,372,1.524590,Agrees,https://www.allsides.com/news-source/verge,left-center,244,The Verge,616
1069913,http://www.npr.org/blogs/thetwo-way/2013/03/21...,0209335-09670257eac5233f00f71dc6a30560b4.txt,npr.org,18286,1.380180,Somewhat Agrees,https://www.allsides.com/news-source/npr-media...,center,13249,NPR News,31535
